# 1. Project Packages

In [37]:
from operator import truediv
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from torchsummary import summary
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
import scipy.io as sio
import torch.nn as nn
import torchvision
import numpy as np
import spectral
import torch
import math
import wget
import time
import os

# 2. Proposed Model

In [38]:
class HybridSN(nn.Module):
    def __init__(self):
        super(HybridSN, self).__init__()
        self.conv3d_1 = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=(7, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(8),
            nn.ReLU(inplace=True),
        )
        self.conv3d_2 = nn.Sequential(
            nn.Conv3d(8, 16, kernel_size=(5, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(16),
            nn.ReLU(inplace=True),
        )
        self.conv3d_3 = nn.Sequential(
            nn.Conv3d(16, 32, kernel_size=(3, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True)
        )

        self.conv2d = nn.Sequential(
            nn.Conv2d(96, 64, kernel_size=(3, 3), stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.fc1 = nn.Linear(3136, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 16)
        self.dropout = nn.Dropout(p=0.4)

    def forward(self, x):
        out = self.conv3d_1(x)
        out = self.conv3d_2(out)
        out = self.conv3d_3(out)
        #print(out.shape)
        #print(out.reshape(out.shape[0], -1, 19, 19).shape)
        out = self.conv2d(out.reshape(out.shape[0], -1, 9, 9))
        out = out.reshape(out.shape[0], -1)
        out = F.relu(self.dropout(self.fc1(out)))
        out = F.relu(self.dropout(self.fc2(out)))
        out = self.fc3(out)
        return out

# 3. Data Preprocessing Modules

## 3.1 PCA for Data Dimensional Reducing

In [39]:
# 对高光谱数据 X 应用 PCA 变换
def applyPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX

## 3.2 Forming Patches of hyperspectral images

In [40]:
# 对单个像素周围提取 patch 时，边缘像素就无法取了，因此，给这部分像素进行 padding 操作
def padWithZeros(X, margin=2):
    newX = np.zeros(
        (X.shape[0] + 2 * margin, X.shape[1] + 2 * margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels=True):
    # 给 X 做 padding
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # 获得 y 中的标记样本数---10249
    count = 0
    for r in range(0, y.shape[0]):
        for c in range(0, y.shape[1]):
            if y[r, c] != 0:
                count = count + 1

    # split patches
    patchesData = np.zeros([count, windowSize, windowSize, X.shape[2]])
    patchesLabels = np.zeros(count)

    count = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            if y[r - margin, c - margin] != 0:
                patch = zeroPaddedX[r - margin:r + margin + 1,
                                    c - margin:c + margin + 1]
                patchesData[count, :, :, :] = patch
                patchesLabels[count] = y[r - margin, c - margin]
                count = count + 1

    return patchesData, patchesLabels

## 3.3 Splitting Dataset into trainSet & testingSet 

In [41]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=testRatio, random_state=randomState, stratify=y)
    return X_train, X_test, y_train, y_test

## 3.4 Data Loading

In [42]:
def loadData(name):
    data_path = "D:\VsCode WorkSpace\Hybrid2D&3D\Data"
    if name == 'IP':
        data = sio.loadmat(
            os.path.join(data_path, 'Indian-Pines\Indian_pines_corrected.mat')
        )['indian_pines_corrected']
        labels = sio.loadmat(
            os.path.join(
                data_path,
                'Indian-Pines\Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(
            os.path.join(data_path,
                         'Salinas\Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(
            data_path, 'Salinas\Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(
            os.path.join(data_path, 'Pavia-University\PaviaU.mat'))['paviaU']
        labels = sio.loadmat(
            os.path.join(data_path,
                         'Pavia-University\PaviaU_gt.mat'))['paviaU_gt']
    else:
        print("NO DATASET")
        exit()
    return data, labels

## 3.5 Focal Loss

In [43]:
def focal_loss(input_values, gamma):
    """Computes the focal loss"""
    p = torch.exp(-input_values)
    #loss = (1 - p) ** gamma * input_values
    loss = (1 - p)**gamma * input_values * 10
    return loss.mean()


class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=0.):
        super(FocalLoss, self).__init__()
        assert gamma >= 0
        self.gamma = gamma
        self.weight = weight

    def forward(self, input, target):
        return focal_loss(
            F.cross_entropy(input,
                            target,
                            reduction='none',
                            weight=self.weight), self.gamma)

# 4. Training Samples Initialization

In [44]:
name = "IP"
if (name == "IP" or name == "SA"):
    class_num = 16
elif (name == "PU"):
    class_num = 9

# 用于测试样本的比例
test_ratio = 0.95
# 每个像素周围提取 patch 的尺寸
patch_size = 15
# 使用 PCA 降维，得到主成分的数量
pca_components = 15
X, y = loadData(name)

""" testing set """
class TestDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtest.shape[0]
        self.x_data = torch.FloatTensor(Xtest)
        self.y_data = torch.LongTensor(ytest)

    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len

""" training set """
class TrainDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtrain.shape[0]
        self.x_data = torch.FloatTensor(Xtrain)
        self.y_data = torch.LongTensor(ytrain)

    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len

print('Hyperspectral data shape: ', X.shape)  
print('Label shape: ', y.shape)  

print('\n... ... PCA tranformation ... ...')
X_pca = applyPCA(X, numComponents=pca_components)
print('Data shape after PCA: ', X_pca.shape)  

print('\n... ... create data cubes ... ...')
X_pca, y = createImageCubes(
    X_pca, y, windowSize=patch_size) 
print('Data cube X shape: ', X_pca.shape)  
print('Data cube y shape: ', y.shape)  

print('\n... ... create train & test data ... ...')
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X_pca, y, test_ratio)
print('Xtrain shape: ', Xtrain.shape)  
print('Xtest  shape: ', Xtest.shape)  

# 改变 Xtrain, Ytrain 的形状，以符合 keras 的要求
Xtrain = Xtrain.reshape(-1, patch_size, patch_size, pca_components, 1)
Xtest = Xtest.reshape(-1, patch_size, patch_size, pca_components, 1)
print('before transpose: Xtrain shape: ', Xtrain.shape)  
print('before transpose: Xtest  shape: ', Xtest.shape)  

# 为了适应 pytorch 结构，数据要做 transpose
Xtrain = Xtrain.transpose(0, 4, 3, 1, 2)
Xtest = Xtest.transpose(0, 4, 3, 1, 2)
print('after transpose: Xtrain shape: ', Xtrain.shape)  
print('after transpose: Xtest  shape: ', Xtest.shape)  
"""在此之前都是对数据的预处理"""
###########数据加载loader############
# 创建 trainloader 和 testloader
trainset = TrainDS()
testset = TestDS()
train_loader = torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=53,  # 128,53
    shuffle=True,
    num_workers=0)
test_loader = torch.utils.data.DataLoader(
    dataset=testset,
    batch_size=53,  # 128,53
    shuffle=False,
    num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
one = torch.ones(53, dtype=torch.long).to(device)
if test_ratio == 0.95:
    two = torch.ones(35, dtype=torch.long).to(device)
elif test_ratio == 0.90:
    two = torch.ones(17, dtype=torch.long).to(device)
elif test_ratio == 0.85:
    two = torch.ones(55, dtype=torch.long).to(device)

# 网络放到GPU上,一些metric的设定
net = HybridSN().to(device)
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

Hyperspectral data shape:  (145, 145, 200)
Label shape:  (145, 145)

... ... PCA tranformation ... ...
Data shape after PCA:  (145, 145, 15)

... ... create data cubes ... ...
Data cube X shape:  (10249, 15, 15, 15)
Data cube y shape:  (10249,)

... ... create train & test data ... ...
Xtrain shape:  (512, 15, 15, 15)
Xtest  shape:  (9737, 15, 15, 15)
before transpose: Xtrain shape:  (512, 15, 15, 15, 1)
before transpose: Xtest  shape:  (9737, 15, 15, 15, 1)
after transpose: Xtrain shape:  (512, 1, 15, 15, 15)
after transpose: Xtest  shape:  (9737, 1, 15, 15, 15)


# 5. Training

In [45]:
# 训练开始时间
start_time = time.time()

net.train()
total_loss = 0
# proc_bar = tqdm(range(150))
for epoch in range(150):
    # proc_bar.set_description(f"正处于第{epoch}回合：")
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        try:
            labels = labels - one
        except:
            labels = labels - two
        #print(labels)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('[Epoch: %d]   [loss avg: %.4f]   [current loss: %.4f]' %
          (epoch + 1, total_loss / (epoch + 1), loss.item()))

# proc_bar.close()

[Epoch: 1]   [loss avg: 232.9116]   [current loss: 19.5727]
[Epoch: 2]   [loss avg: 199.6020]   [current loss: 14.7035]
[Epoch: 3]   [loss avg: 177.2983]   [current loss: 12.3985]
[Epoch: 4]   [loss avg: 159.1427]   [current loss: 12.4088]
[Epoch: 5]   [loss avg: 145.2855]   [current loss: 7.9955]
[Epoch: 6]   [loss avg: 132.7542]   [current loss: 5.7663]
[Epoch: 7]   [loss avg: 122.2476]   [current loss: 5.4468]
[Epoch: 8]   [loss avg: 113.4562]   [current loss: 6.0857]
[Epoch: 9]   [loss avg: 105.2844]   [current loss: 2.3983]
[Epoch: 10]   [loss avg: 98.1324]   [current loss: 2.7416]
[Epoch: 11]   [loss avg: 92.0303]   [current loss: 2.5654]
[Epoch: 12]   [loss avg: 86.0286]   [current loss: 0.9840]
[Epoch: 13]   [loss avg: 80.7570]   [current loss: 0.8448]
[Epoch: 14]   [loss avg: 76.2973]   [current loss: 1.9628]
[Epoch: 15]   [loss avg: 72.3852]   [current loss: 0.6739]
[Epoch: 16]   [loss avg: 68.8911]   [current loss: 0.8152]
[Epoch: 17]   [loss avg: 65.6426]   [current loss: 0

## 5.1 Training time

In [46]:
end_time = time.time()
print(f"time cost:{(end_time-start_time)/60} min")
print('Finished Training')

time cost:0.9628042141596477 min
Finished Training


# 6. Saving & Outputing the model params of proposed model

In [47]:
# 指定模型保存的地址
path = r'D:\VsCode WorkSpace\FEHN-FL\Assets\HybridSN\IP-Patch15\model.pth'
torch.save(net, path)
# 输出模型参数
summary(net, (1, 15, 15, 15))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1         [-1, 8, 9, 13, 13]             512
       BatchNorm3d-2         [-1, 8, 9, 13, 13]              16
              ReLU-3         [-1, 8, 9, 13, 13]               0
            Conv3d-4        [-1, 16, 5, 11, 11]           5,776
       BatchNorm3d-5        [-1, 16, 5, 11, 11]              32
              ReLU-6        [-1, 16, 5, 11, 11]               0
            Conv3d-7          [-1, 32, 3, 9, 9]          13,856
       BatchNorm3d-8          [-1, 32, 3, 9, 9]              64
              ReLU-9          [-1, 32, 3, 9, 9]               0
           Conv2d-10             [-1, 64, 7, 7]          55,360
      BatchNorm2d-11             [-1, 64, 7, 7]             128
             ReLU-12             [-1, 64, 7, 7]               0
           Linear-13                  [-1, 256]         803,072
          Dropout-14                  [

# 7. Evaluating the proposed model

In [48]:
a = np.ones(Xtest.shape[0])  ##9225是Xtest.shape[0]
ytest = ytest - a

count = 0
# 模型测试
net.eval()
for inputs, _ in test_loader:
    inputs = inputs.to(device)
    outputs = net(inputs)
    outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
    if count == 0:
        y_pred_test = outputs
        count = 1
    else:
        y_pred_test = np.concatenate((y_pred_test, outputs))

# 8. Generating the report of evaluation

## 8.1 text report

In [49]:
# 生成分类报告
classification = classification_report(ytest, y_pred_test, digits=4)
print(classification)


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc


def reports(test_loader, y_test, name):
    count = 0
    # 模型测试
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = net(inputs)
        outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        if count == 0:
            y_pred = outputs
            count = 1
        else:
            y_pred = np.concatenate((y_pred, outputs))

    if name == 'IP':
        target_names = [
            'Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn', 'Grass-pasture',
            'Grass-trees', 'Grass-pasture-mowed', 'Hay-windrowed', 'Oats',
            'Soybean-notill', 'Soybean-mintill', 'Soybean-clean', 'Wheat',
            'Woods', 'Buildings-Grass-Trees-Drives', 'Stone-Steel-Towers'
        ]
    elif name == 'SA':
        target_names = [
            'Brocoli_green_weeds_1', 'Brocoli_green_weeds_2', 'Fallow',
            'Fallow_rough_plow', 'Fallow_smooth', 'Stubble', 'Celery',
            'Grapes_untrained', 'Soil_vinyard_develop',
            'Corn_senesced_green_weeds', 'Lettuce_romaine_4wk',
            'Lettuce_romaine_5wk', 'Lettuce_romaine_6wk',
            'Lettuce_romaine_7wk', 'Vinyard_untrained',
            'Vinyard_vertical_trellis'
        ]
    elif name == 'PU':
        target_names = [
            'Asphalt', 'Meadows', 'Gravel', 'Trees', 'Painted metal sheets',
            'Bare Soil', 'Bitumen', 'Self-Blocking Bricks', 'Shadows'
        ]

    classification = classification_report(y_test,
                                           y_pred,
                                           target_names=target_names)
    oa = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(y_test, y_pred)

    return classification, confusion, oa * 100, each_acc * 100, aa * 100, kappa * 100


# 将结果写在文件里
classification, confusion, oa, each_acc, aa, kappa = reports(
    test_loader, ytest, name)
classification = str(classification)
confusion = str(confusion)
file_name = r"D:\VsCode WorkSpace\FEHN-FL\Assets\HybridSN\IP-Patch15\classification_report(0.95-loss).txt"

with open(file_name, 'w') as x_file:
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('{} training time (min)'.format((end_time-start_time)/60))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

              precision    recall  f1-score   support

         0.0     0.7143    0.3409    0.4615        44
         1.0     0.9747    0.9071    0.9397      1357
         2.0     0.9199    0.9747    0.9465       789
         3.0     0.9888    0.7822    0.8734       225
         4.0     0.9868    0.9739    0.9803       459
         5.0     0.9957    0.9942    0.9949       693
         6.0     1.0000    0.5926    0.7442        27
         7.0     0.9303    1.0000    0.9639       454
         8.0     1.0000    0.6842    0.8125        19
         9.0     0.9372    0.9697    0.9531       923
        10.0     0.9461    0.9867    0.9660      2332
        11.0     0.9445    0.9076    0.9257       563
        12.0     1.0000    1.0000    1.0000       195
        13.0     0.9622    0.9967    0.9792      1202
        14.0     0.9724    0.8638    0.9149       367
        15.0     0.9880    0.9318    0.9591        88

    accuracy                         0.9560      9737
   macro avg     0.9538   

## 8.2 images of prediction report

In [50]:
# 显示结果
# load the original image

# X, y = loadData(name)

# height = y.shape[0]
# width = y.shape[1]

# X = applyPCA(X, numComponents=pca_components)
# X = padWithZeros(X, patch_size // 2)

# # 逐像素预测类别
# outputs = np.zeros((height, width))
# for i in range(height):
#     for j in range(width):
#         if int(y[i, j]) == 0:
#             continue
#         else:
#             image_patch = X[i:i + patch_size, j:j + patch_size, :]
#             image_patch = image_patch.reshape(1, image_patch.shape[0],
#                                               image_patch.shape[1],
#                                               image_patch.shape[2], 1)
#             X_test_image = torch.FloatTensor(
#                 image_patch.transpose(0, 4, 3, 1, 2)).to(device)
#             prediction = net(X_test_image)
#             prediction = np.argmax(prediction.detach().cpu().numpy(), axis=1)
#             outputs[i][j] = prediction + 1
#     if i % 20 == 0:
#         print('... ... row ', i, ' handling ... ...')

# oringal_image = spectral.imshow(classes=y, figsize=(7, 7))
# predict_image = spectral.imshow(classes=outputs.astype(int), figsize=(7, 7))
# spectral.save_rgb("Assets\FE-NET\(PU-3-3-0.85)FE-NET-原始.jpg",
#                   y.astype(int),
#                   colors=spectral.spy_colors)
# spectral.save_rgb("Assets\FE-NET\(PU-3-3-0.85)FE-NET-预测.jpg",
#                   outputs.astype(int),
#                   colors=spectral.spy_colors)